In [ ]:
from nltk.tokenize import word_tokenize
import json
import re
import nltk
from torch.utils.data import Dataset, DataLoader
import torch
import gensim.downloader as api
import torch.nn as nn
import torch.optim as optim
import numpy as np
from conlleval import evaluate
import matplotlib.pyplot as plt

label_to_index = {'B':0, 'I':1, 'O':2}
def preprocess_data(data):
    preprocessed_data = []
    
    for item in data:
        aspects = []
        for aspect in item['aspect_terms']:
            aspects.append(aspect['term'])

        sentence = item['sentence']
        tokens = word_tokenize(sentence)
        length = len(tokens)
        labels = ['O'] * length

        for aspect in item['aspect_terms']:
            start_index = int(aspect['from'])
            end_index = int(aspect['to'])
            aspect_part = sentence[start_index:end_index+1]
            aspect_tokens = word_tokenize(aspect_part)
            aspect_length = len(aspect_tokens)

            for i in range(length):
                if tokens[i] == aspect_tokens[0]:
                    labels[i] = 'B'
                    for j in range(aspect_length-1):
                        labels[i+j+1] = 'I'
                    break

        preprocessed_data .append(
            {
                'sentence' : item['sentence'],
                'tokens' : tokens,
                'labels' : labels,
                'aspect_terms' : aspects,
            }
        )
    return preprocessed_data

def get_embedding(data,embedding_model):
    embedded_data = []
    
    for item in data:
        tokens = item['tokens']
        labels = item['labels']
        word_embedding = []
        new_labels = []
        for i in range(len(tokens)):
            if tokens[i] in embedding_model:
                word_embedding.append(embedding_model[tokens[i]])
                new_labels.append(labels[i])
                
        embedded_data.append(
            {
            'sentence' : item['sentence'],
            'tokens' : tokens,
            'embeddings' : word_embedding,
            'labels' : new_labels,
            'aspect_terms' : item['aspect_terms']
            }
        )
    return embedded_data

class custom_dataset(Dataset):
    def __init__(self,data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        item = self.data[idx]
        embeddings = np.array(item['embeddings'], dtype=np.float32)
        labels = np.array(item['labels'], dtype=np.int64)
        labels = labels[:len(embeddings)]
        return torch.tensor(embeddings), torch.tensor(labels)

def label_indexing(data, label_to_index):
    new_data = []
    for item in data:
        
        label_index = []
        for label in item['labels']:
            label_index.append(label_to_index[label])


        new_entry = {
            'sentence': item['sentence'],
            'tokens': item['tokens'],
            'labels': label_index,
            'aspect_terms': item['aspect_terms']
        }
        new_data.append(new_entry)

        
    return new_data

import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device) 

        out, _ = self.gru(x, h0)  

        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc_out(out)  
        return out

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)

        
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)  

        out, _ = self.rnn(x, h0)  

       
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc_out(out)  
        return out
        
lr = 0.001

def evaluate_test_data(best_model_path, test_file_path,flag):
    with open(test_file_path, 'r') as file:
        test_data = json.load(file)

    test_data_processed = preprocess_data(test_data)
    test = label_indexing(test_data_processed,label_to_index)
    
    best_model = torch.load('C:/Users/rites/Downloads/best_model.pt', weights_only=False)

    def format_for_conlleval(labels, predictions):
        return [f"w{idx} O {true} {pred}" for idx, (true, pred) in enumerate(zip(labels, predictions))]
        
    if flag == True:
        glove_model = api.load("glove-wiki-gigaword-300")
        
        test1 = get_embedding(test, glove_model)
        
        print(test1)
    
        test_dataset = custom_dataset(test1)
    
        print(test_dataset)

        max1 = 0
        for item in test_dataset:
            max1 = max(len(item[0]),max1)
        
        new_test_dataset = []
    
        for item in test_dataset:
    
            
            zero_rows = torch.zeros((max1 - len(item[0]), 300))  
            padded_embeddings = torch.cat((item[0], zero_rows), dim=0)
        
            padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
        
            new_test_dataset.append((padded_embeddings, padded_labels))
    
        test_dataset = new_test_dataset
        
        test_loader = DataLoader(test_dataset, batch_size=32)
        
        best_model.eval()  
        overall_test_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            for embeddings, labels in test_loader:
                outputs = best_model(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_test_loss += loss

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                
        print(f1_sum/len(test_loader))
        
    if flag == False:
        glove_model = api.load("glove-wiki-gigaword-300")
    
        test1 = get_embedding(test, glove_model)
        
        print(test1)
    
        test_dataset = custom_dataset(test1)
    
        print(test_dataset)

        max1 = 0
        for item in test_dataset:
            max1 = max(len(item[0]),max1)
        
        new_test_dataset = []
    
        for item in test_dataset:
            length = len(item[0])
            
            
            zero_rows = torch.zeros((max1 - length, 300))  
            padded_embeddings = torch.cat((item[0], zero_rows), dim=0)
        
            # Pad labels with -1
            padded_labels = torch.cat((item[1], torch.full((max1 - length,), -1, dtype=torch.long)))
        
            new_test_dataset.append((padded_embeddings, padded_labels))
    
        test_dataset = new_test_dataset
        
        test_loader = DataLoader(test_dataset, batch_size=32)

        best_model.eval()  
        overall_test_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            for embeddings, labels in test_loader:
                outputs = best_model(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_val_loss += loss

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                
        print(f1_sum/len(test_loader))
    
flag = False
epoch_cycle = [1,2,3,4,5]

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    with open('C:/Users/rites/Downloads/train.json', 'r') as file:
        train_data = json.load(file)
    
    with open('C:/Users/rites/Downloads/val.json', 'r') as file:
        val_data = json.load(file)

    with open('C:/Users/rites/Downloads/train_task_1.json', 'w') as file:
        json.dump(preprocess_data(train_data),file)

    with open('C:/Users/rites/Downloads/val_task_1.json', 'w') as file:
        json.dump(preprocess_data(val_data),file)

    with open('C:/Users/rites/Downloads/train_task_1.json', 'r') as file:
        train_data_processed = json.load(file)
    
    with open('C:/Users/rites/Downloads/val_task_1.json', 'r') as file:
        val_data_processed = json.load(file)
        
    train = label_indexing(train_data_processed,label_to_index)
    val = label_indexing(val_data_processed,label_to_index)

    # model 1 -> rnn + glove
    glove_model = api.load("glove-wiki-gigaword-300")
    
    train1 = get_embedding(train, glove_model)
    val1 = get_embedding(val,glove_model)
    
    print(train1)
    print(val1)

    train_dataset = custom_dataset(train1)
    val_dataset = custom_dataset(val1)

    print(train_dataset)
    print(val_dataset)

    max1 = 0
    for item in train_dataset:
        max1 = max(len(item[0]),max1)
    
    new_train_dataset = []

    for item in train_dataset:
     
        
       

        padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300)) ), dim=0)
    
        
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_train_dataset.append((padded_embeddings, padded_labels))

    train_dataset = new_train_dataset
    
    train_loader = DataLoader(train_dataset, batch_size=32)
    
    max1 = 0
    for item in val_dataset:
        max1 = max(len(item[0]),max1)
    
    new_val_dataset = []

    for item in val_dataset:
    
        
     
        padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300)) ), dim=0)
    
      
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_val_dataset.append((padded_embeddings, padded_labels))

    val_dataset = new_val_dataset

    val_loader = DataLoader(val_dataset, batch_size=32)

    rnn_model1 = RNNModel(input_dim=300, hidden_dim=128, output_dim=3)

    optimizer = optim.Adam(rnn_model1.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-1) 
    
    f1_score_models = []
    f1_chunk_score_models = []
    f1_tag_score_models = []
    num_epochs = 5
    train_loss = []
    val_loss = []
    epoch_f1 = []
    epoch_f1_chunk = []
    epoch_f1_tag = []
    idx_to_bio = {0: 'B', 1: 'I', 2: 'O'}

    
    
    def format_for_conlleval(labels, predictions):
        return [f"w{idx} O {true} {pred}" for idx, (true, pred) in enumerate(zip(labels, predictions))]

    
    for epoch in range(num_epochs):
        rnn_model1.train()
        overall_train_loss = 0
        for embeddings, labels in train_loader:
            
            optimizer.zero_grad()
            outputs = rnn_model1(embeddings)
            
            outputs = outputs.view(-1, outputs.shape[-1])
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            overall_train_loss += loss.item()
            
        train_loss.append(overall_train_loss/len(train_loader))

        rnn_model1.eval()  
        overall_val_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in val_loader:
                outputs = rnn_model1(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_val_loss += loss.item()

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                
        epoch_f1_chunk.append(f1_sum_chunk/len(val_loader))
        epoch_f1_tag.append(f1_sum_tag/len(val_loader))        
        epoch_f1.append(f1_sum/len(val_loader))
        
        val_loss.append(overall_val_loss/len(val_loader))

    print("RNN + Glove")
    print("f1",np.mean(epoch_f1))
    print("f1_chunk",np.mean(epoch_f1_chunk))
    print("f1_tag",np.mean(epoch_f1_tag))
    
    f1_score_models.append(np.mean(epoch_f1))
    f1_chunk_score_models.append(np.mean(epoch_f1_chunk))
    f1_tag_score_models.append(np.mean(epoch_f1_tag))
    
    plt.plot(epoch_cycle, val_loss)
    plt.plot(epoch_cycle,train_loss)
    
    plt.xlabel('epoch')
    plt.ylabel('losses')
    
    plt.savefig('plot1.png') 
    plt.close()
    
    # model2 -> gru + glove
    glove_model = api.load("glove-wiki-gigaword-300")
    
    train1 = get_embedding(train, glove_model)
    val1 = get_embedding(val,glove_model)
    
    print(train1)
    print(val1)

    train_dataset = custom_dataset(train1)
    val_dataset = custom_dataset(val1)

    print(train_dataset)
    print(val_dataset)

    max1 = 0
    for item in train_dataset:
        max1 = max(len(item[0]),max1)

    new_train_dataset = []
    
    for item in train_dataset:
        
        padded_embeddings = torch.cat((item[0],  torch.zeros((max1 - len(item[0]), 300))), dim=0)
    
        
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_train_dataset.append((padded_embeddings, padded_labels))

    train_dataset = new_train_dataset

    train_loader = DataLoader(train_dataset, batch_size=32)

    new_val_dataset = []

    for item in val_dataset:
        

        zero_rows = torch.zeros((max1 - len(item[0]), 300))  
        padded_embeddings = torch.cat((item[0], zero_rows), dim=0)
    
        
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_val_dataset.append((padded_embeddings, padded_labels))

    val_dataset = new_val_dataset

    val_loader = DataLoader(val_dataset, batch_size=32)

    gru_model1 = GRUModel(input_dim=300, hidden_dim=128, output_dim=3)

    optimizer = optim.Adam(gru_model1.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-1) 

    num_epochs = 5
    train_loss = []
    val_loss = []
    epoch_f1 = []
    epoch_f1_tag = []
    epoch_f1_chunk = []
    for epoch in range(num_epochs):
        gru_model1.train()
        overall_train_loss = 0
        for embeddings, labels in train_loader:
            
            optimizer.zero_grad()
            outputs = gru_model1(embeddings)
            
            outputs = outputs.view(-1, outputs.shape[-1])
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            overall_train_loss += loss.item()
            
        train_loss.append(overall_train_loss/len(train_loader))

        gru_model1.eval()  
        overall_val_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in val_loader:
                outputs = gru_model1(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_val_loss += loss.item()

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                
        epoch_f1_chunk.append(f1_sum_chunk/len(val_loader))
        epoch_f1_tag.append(f1_sum_tag/len(val_loader))
        epoch_f1.append(f1_sum/len(val_loader))
        
        val_loss.append(overall_val_loss/len(val_loader))

    plt.plot(epoch_cycle, val_loss)
    plt.plot(epoch_cycle,train_loss)
    
    plt.xlabel('epoch')
    plt.ylabel('losses')
    
    plt.savefig('plot2.png') 
    plt.close()
    
    print("GRU + Glove")
    print("f1",np.mean(epoch_f1))
    print("f1_chunk",np.mean(epoch_f1_chunk))
    print("f1_tag",np.mean(epoch_f1_tag))
    
    f1_score_models.append(np.mean(epoch_f1))
    f1_chunk_score_models.append(np.mean(epoch_f1_chunk))
    f1_tag_score_models.append(np.mean(epoch_f1_tag))

    fasttext_model = api.load("fasttext-wiki-news-subwords-300")
    
    train1 = get_embedding(train, fasttext_model)
    val1 = get_embedding(val,fasttext_model)
    
    print(train1)
    print(val1)

    train_dataset = custom_dataset(train1)
    val_dataset = custom_dataset(val1)

    print(train_dataset)
    print(val_dataset)

    max1 = 0
    for item in train_dataset:
        max1 = max(len(item[0]),max1)

    new_train_dataset = []
    
    for item in train_dataset:
       
        padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300))  ), dim=0)
    
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_train_dataset.append((padded_embeddings, padded_labels))

    train_dataset = new_train_dataset

    train_loader = DataLoader(train_dataset, batch_size=32)

    new_val_dataset = []

    for item in val_dataset:
     
       
        
        padded_embeddings = torch.cat((item[0], torch.zeros((max1 -  len(item[0]), 300))  ), dim=0)
    
        # Pad labels with -1
        padded_labels = torch.cat((item[1], torch.full((max1 -  len(item[0]),), -1, dtype=torch.long)))
    
        new_val_dataset.append((padded_embeddings, padded_labels))

    val_dataset = new_val_dataset

    val_loader = DataLoader(val_dataset, batch_size=32)

    gru_model2 = GRUModel(input_dim=300, hidden_dim=128, output_dim=3)

    optimizer = optim.Adam(gru_model2.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-1) 

    num_epochs = 5
    train_loss = []
    val_loss = []
    epoch_f1 = []
    epoch_f1_tag = []
    epoch_f1_chunk = []
    for epoch in range(num_epochs):
        gru_model2.train()
        overall_train_loss = 0
        for embeddings, labels in train_loader:
            
            optimizer.zero_grad()
            outputs = gru_model2(embeddings)
            
            outputs = outputs.view(-1, outputs.shape[-1])
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            overall_train_loss += loss.item()
            
        train_loss.append(overall_train_loss/len(train_loader))

        gru_model2.eval()  
        overall_val_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in val_loader:
                outputs = gru_model2(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_val_loss += loss.item()

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                
        epoch_f1_chunk.append(f1_sum_chunk/len(val_loader))
        epoch_f1_tag.append(f1_sum_tag/len(val_loader))
        epoch_f1.append(f1_sum/len(val_loader))
        
        val_loss.append(overall_val_loss/len(val_loader))

    print("GRU + Fasttext")
    print("f1",np.mean(epoch_f1))
    print("f1_chunk",np.mean(epoch_f1_chunk))
    print("f1_tag",np.mean(epoch_f1_tag))
    
    f1_score_models.append(np.mean(epoch_f1))
    f1_chunk_score_models.append(np.mean(epoch_f1_chunk))
    f1_tag_score_models.append(np.mean(epoch_f1_tag))
    
    plt.plot(epoch_cycle, val_loss)
    plt.plot(epoch_cycle,train_loss)
    
    plt.xlabel('epoch')
    plt.ylabel('losses')
    
    plt.savefig('plot3.png') 
    plt.close()
    
    # model 4 -> rnn + fastext
    fasttext_model = api.load("fasttext-wiki-news-subwords-300")
    
    train1 = get_embedding(train, fasttext_model)
    val1 = get_embedding(val,fasttext_model)
    
    print(train1)
    print(val1)

    train_dataset = custom_dataset(train1)
    val_dataset = custom_dataset(val1)

    print(train_dataset)
    print(val_dataset)

    max1 = 0
    for item in train_dataset:
        max1 = max(len(item[0]),max1)

    new_train_dataset = []
    
    for item in train_dataset:
       
        padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300))), dim=0)
    
       
        padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
    
        new_train_dataset.append((padded_embeddings, padded_labels))

    train_dataset = new_train_dataset

    train_loader = DataLoader(train_dataset, batch_size=32)

    new_val_dataset = []

    for item in val_dataset:
        length = len(item[0])
        
        zero_rows = torch.zeros((max1 - length, 300))  
        padded_embeddings = torch.cat((item[0], zero_rows), dim=0)
    
        padded_labels = torch.cat((item[1], torch.full((max1 - length,), -1, dtype=torch.long)))
    
        new_val_dataset.append((padded_embeddings, padded_labels))

    val_dataset = new_val_dataset

    val_loader = DataLoader(val_dataset, batch_size=32)

    rnn_model2 = RNNModel(input_dim=300, hidden_dim=128, output_dim=3)

    optimizer = optim.Adam(rnn_model2.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=-1) 

    num_epochs = 5
    train_loss = []
    val_loss = []
    epoch_f1 = []
    epoch_f1_tag = []
    epoch_f1_chunk = []
    for epoch in range(num_epochs):
        rnn_model2.train()
        overall_train_loss = 0
        for embeddings, labels in train_loader:
            
            optimizer.zero_grad()
            outputs = rnn_model2(embeddings)
            
            outputs = outputs.view(-1, outputs.shape[-1])
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            overall_train_loss += loss.item()
            
        train_loss.append(overall_train_loss/len(train_loader))

        rnn_model2.eval()  
        overall_val_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in val_loader:
                outputs = rnn_model2(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                overall_val_loss += loss.item()

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum += (chunk_f1 + tag_f1)/2
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                
        epoch_f1_chunk.append(f1_sum_chunk/len(val_loader))
        epoch_f1_tag.append(f1_sum_tag/len(val_loader))
        epoch_f1.append(f1_sum/len(val_loader))
        
        val_loss.append(overall_val_loss/len(val_loader))

    print("RNN + Fasttext")
    print("f1",np.mean(epoch_f1))
    print("f1_chunk",np.mean(epoch_f1_chunk))
    print("f1_tag",np.mean(epoch_f1_tag))
    
    f1_score_models.append(np.mean(epoch_f1))
    f1_chunk_score_models.append(np.mean(epoch_f1_chunk))
    f1_tag_score_models.append(np.mean(epoch_f1_tag))
    
    plt.plot(epoch_cycle, val_loss)
    plt.plot(epoch_cycle,train_loss)
    
    plt.xlabel('epoch')
    plt.ylabel('losses')
    
    plt.savefig('plot4.png') 
    plt.close()
    
    max_index = f1_score_models.index(max(f1_score_models)) 

    if max_index == 0:
        torch.save(rnn_model1, 'C:/Users/rites/Downloads/best_model.pt') 
        print("f1_score_models",f1_score_models[max_index])
        print("f1_chunk_score_models",f1_chunk_score_models[max_index])
        print("f1_tag_score_models",f1_tag_score_models[max_index])
        print("rnn + glove")
        flag = True
    if max_index == 1:
        torch.save(gru_model1,'C:/Users/rites/Downloads/best_model.pt') 
        print("f1_score_models",f1_score_models[max_index])
        print("f1_chunk_score_models",f1_chunk_score_models[max_index])
        print("f1_tag_score_models",f1_tag_score_models[max_index])
        print("gru + glove")
        flag = True
    if max_index == 2:
        torch.save(gru_model2,'C:/Users/rites/Downloads/best_model.pt') 
        print("f1_score_models",f1_score_models[max_index])
        print("f1_chunk_score_models",f1_chunk_score_models[max_index])
        print("f1_tag_score_models",f1_tag_score_models[max_index])
        print("gru + fasttext")
    if max_index == 3:
        torch.save(rnn_model2,'C:/Users/rites/Downloads/best_model.pt') 
        print("f1_score_models",f1_score_models[max_index])
        print("f1_chunk_score_models",f1_chunk_score_models[max_index])
        print("f1_tag_score_models",f1_tag_score_models[max_index])
        print("rnn + fasttext")
    
if __name__ == '__main__':
    main()      

Using device: cpu


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



RNN + Glove
f1 0.8147603008421049
f1_chunk 0.6811704213688271
f1_tag 0.9483501803153829


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



GRU + Glove
f1 0.8137980366707451
f1_chunk 0.6794268561818558
f1_tag 0.9481692171596341


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



GRU + Fasttext
f1 0.7836185498143601
f1_chunk 0.6240937762144577
f1_tag 0.9431433234142622


In [3]:
evaluate_test_data('C:/Users/rites/Downloads/best_model.pt', 'C:/Users/rites/Downloads/test.json',flag)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



0.032098687081717994
0.8336598576760306
0.43287927237887425


In [ ]:
idx_to_bio = {0: 'B', 1: 'I', 2: 'O'}
def evaluate_test_data(best_model_path, test_file_path,flag):
    with open(test_file_path, 'r') as file:
        test_data = json.load(file)

    test_data_processed = preprocess_data(test_data)
    test = label_indexing(test_data_processed,label_to_index)
    
    best_model = torch.load(best_model_path, weights_only=False)

    def format_for_conlleval(labels, predictions):
        return [f"w{idx} O {true} {pred}" for idx, (true, pred) in enumerate(zip(labels, predictions))]

        
    if flag == True:
        glove_model = api.load("glove-wiki-gigaword-300")
        
        test1 = get_embedding(test, glove_model)
        
        print(test1)
    
        test_dataset = custom_dataset(test1)
    
        print(test_dataset)

        max1 = 0
        for item in test_dataset:
            max1 = max(len(item[0]),max1)
        
        new_test_dataset = []
    
        for item in test_dataset:
            

            padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300))  ), dim=0)
        
            padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
        
            new_test_dataset.append((padded_embeddings, padded_labels))
    
        test_dataset = new_test_dataset
        
        test_loader = DataLoader(test_dataset, batch_size=32)
        
        best_model.eval()  
        overall_test_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in test_loader:
                outputs = best_model(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                f1_sum += (chunk_f1 + tag_f1)/2

        print(f1_sum_chunk/len(test_loader))
        print(f1_sum_tag/len(test_loader))
        print(f1_sum/len(test_loader))
        
    if flag == False:
        glove_model = api.load("glove-wiki-gigaword-300")
    
        test1 = get_embedding(test, glove_model)
        
        print(test1)
    
        test_dataset = custom_dataset(test1)
    
        print(test_dataset)

        max1 = 0
        for item in test_dataset:
            max1 = max(len(item[0]),max1)
        
        new_test_dataset = []
    
        for item in test_dataset:
      

            padded_embeddings = torch.cat((item[0], torch.zeros((max1 - len(item[0]), 300))  ), dim=0)
        
          
            padded_labels = torch.cat((item[1], torch.full((max1 - len(item[0]),), -1, dtype=torch.long)))
        
            new_test_dataset.append((padded_embeddings, padded_labels))
    
        test_dataset = new_test_dataset
        
        test_loader = DataLoader(test_dataset, batch_size=32)

        best_model.eval()  
        overall_test_loss = 0

        with torch.no_grad():  
            f1_sum = 0
            f1_sum_chunk = 0
            f1_sum_tag = 0
            for embeddings, labels in test_loader:
                outputs = best_model(embeddings)
                
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)

                final_outputs = []
                final_labels = []
                for i in range(len(labels)):
                    if labels[i]!=-1:
                        max_index = torch.argmax(outputs[i])
                        final_outputs.append(idx_to_bio[max_index.item()])
                        final_labels.append(idx_to_bio[labels[i].item()])

                formatted_data = format_for_conlleval(final_labels, final_outputs)

                result = evaluate(formatted_data)

                chunk_f1 = result['overall']['chunks']['evals']['f1']
                tag_f1 = result['overall']['tags']['evals']['f1']
                
                f1_sum_chunk += chunk_f1
                f1_sum_tag += tag_f1
                f1_sum += (chunk_f1 + tag_f1)/2

        print(f1_sum_chunk/len(test_loader))
        print(f1_sum_tag/len(test_loader))
        print(f1_sum/len(test_loader))